# 高阶NumPy

## A.1 ndarray对象内幕

In [1]:
import numpy as np
np.ones((10,5)).shape

(10, 5)

In [3]:
np.ones((3,4,5),dtype=np.float64).strides

(160, 40, 8)

### A.1.1 NumPy dtype 层次结构

dtype有超类，np.integer和np.floating  可以和np.issubdtype函数一起使用(检查dtype是否在类型列表中)

In [4]:
ints=np.ones(10,dtype=np.uint16)
floats=np.ones(10,dtype=np.float32)

In [5]:
np.issubdtype(ints.dtype,np.integer)

True

In [6]:
np.issubdtype(floats.dtype,np.floating)

True

调用类型的mro方法来查看特定dtype的所有父类

In [7]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

In [8]:
np.issubdtype(ints.dtype,np.number)

True

## A.2 高阶数组操作
### A.2.1 重塑数组

向reshape数组实例方法传递一个表示新形状的元组

In [11]:
arr=np.arange(8)
arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [13]:
arr.reshape((4,2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [14]:
arr.reshape((4,2)).reshape((2,4))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

传递的形状维度可以有一个值是-1，表示维度通过数据进行推断

In [21]:
arr=np.arange(15)
arr.reshape((5,-1)) # 展开是5行

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

数组的shape属性是一个元组，它也可以被传递给reshape

In [22]:
other_arr=np.ones((3,5))
other_arr.shape

(3, 5)

In [24]:
arr.reshape(other_arr.shape)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

reshape的反操作可以将更高维度的数组转换为一维数组，这种操作通常被称为扁平化(flattening)或分散化(raveling)

In [27]:
arr=np.arange(15).reshape(5,3)
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [30]:
arr.ravel() # 转换为一维数组

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [31]:
arr.flatten() # 转换为一维数组

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [35]:
id(arr)

2523272838080

In [36]:
id(arr.ravel())

2523272838800

In [37]:
id(arr.flatten())

2523272837200

### A.2.2 C顺序和Fortran顺序
默认情况下，NumPy数组是按行方向顺序建立的

行和列方向的顺序也分别称为C顺序和Fortran顺序

像reshape和ravel函数接收一个order参数，该参数表示数据在数组中使用哪种顺序，在大部分情况下，该参数可以被设置为'C'和'F'(还有一些不常用的选项'A'和'K')

In [39]:
arr=np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [40]:
arr.ravel() # 转换为一维数组

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [44]:
arr.ravel(order='F') # 或者arr.ravel('F')
# 表示按列转换为一维数组

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

### A.2.3 连接和分隔数组
numpy.concatenate可以获取数组的序列(元组，列表等)，并沿着输入轴将它们按顺序连接在一起

In [48]:
arr1=np.array([[1,2,3],[4,5,6]])
arr2=np.array([[7,8,9],[10,11,12]])
np.concatenate([arr1,arr2],axis=0) # 按axis=0 即轴0连接
# 等价于 np.vstack((arr1,arr2)) 沿axis=0连接

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [47]:
np.concatenate([arr1,arr2],axis=1) # 按axis=1 即轴1连接
# 等价于 np.hstack((arr1,arr2)) 沿axis=1连接

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

split可以将一个数组沿轴向切片成多个数组

In [51]:
arr=np.random.randn(5,2)
arr

array([[ 1.35361212,  0.35637851],
       [-1.33355331, -0.21076449],
       [ 0.54955148,  2.09761194],
       [ 0.54215529, -0.32406815],
       [-1.5418364 , -0.74524633]])

In [54]:
first,second,third=np.split(arr,[1,3]) # 在索引1和3的位置分别拆分

In [55]:
first

array([[1.35361212, 0.35637851]])

In [56]:
second

array([[-1.33355331, -0.21076449],
       [ 0.54955148,  2.09761194]])

In [57]:
third

array([[ 0.54215529, -0.32406815],
       [-1.5418364 , -0.74524633]])

#### A.2.3.1 堆叠助手：r_和c_

In [59]:
arr=np.arange(6)
arr1=arr.reshape((3,2))
# array([[0, 1],
#       [2, 3],
#      [4, 5]])
arr2=np.random.randn(3,2)
# array([[-0.94523653, -1.14963026],
#       [-0.89861386,  0.20847323],
#       [ 0.20358264, -0.1620006 ]])

In [60]:
np.r_[arr1,arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [-0.94523653, -1.14963026],
       [-0.89861386,  0.20847323],
       [ 0.20358264, -0.1620006 ]])

In [61]:
np.c_[arr1,arr2]

array([[ 0.        ,  1.        , -0.94523653, -1.14963026],
       [ 2.        ,  3.        , -0.89861386,  0.20847323],
       [ 4.        ,  5.        ,  0.20358264, -0.1620006 ]])

### A.2.4 重复元素：tile和repeat

In [64]:
arr=np.arange(3)
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

如果传递一个整数，每个元素都会复制相应的次数；如果你传递了一个整数数组，每个元素都会重复相应的不同次数

In [65]:
arr.repeat([2,3,4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

多维数组可以在指定的轴向上对它们的元素进行重复

In [66]:
arr=np.random.randn(2,2)
arr

array([[ 1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 ]])

In [67]:
arr.repeat(2,axis=0) # 在0轴进行重复

array([[ 1.02284631,  0.32243496],
       [ 1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 ],
       [-0.58767126,  0.3832482 ]])

In [68]:
arr.repeat([2,3],axis=0)

array([[ 1.02284631,  0.32243496],
       [ 1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 ],
       [-0.58767126,  0.3832482 ],
       [-0.58767126,  0.3832482 ]])

In [69]:
arr.repeat([2,3],axis=1)

array([[ 1.02284631,  1.02284631,  0.32243496,  0.32243496,  0.32243496],
       [-0.58767126, -0.58767126,  0.3832482 ,  0.3832482 ,  0.3832482 ]])

In [70]:
arr

array([[ 1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 ]])

In [71]:
np.tile(arr,2)

array([[ 1.02284631,  0.32243496,  1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 , -0.58767126,  0.3832482 ]])

In [78]:
np.tile(arr,(2,2)) # 整体按行为2行列为1行进行复制

array([[ 1.02284631,  0.32243496,  1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 , -0.58767126,  0.3832482 ],
       [ 1.02284631,  0.32243496,  1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 , -0.58767126,  0.3832482 ]])

In [75]:
np.tile(arr,(3,2)) # 整体按行为3行列为2行进行复制

array([[ 1.02284631,  0.32243496,  1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 , -0.58767126,  0.3832482 ],
       [ 1.02284631,  0.32243496,  1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 , -0.58767126,  0.3832482 ],
       [ 1.02284631,  0.32243496,  1.02284631,  0.32243496],
       [-0.58767126,  0.3832482 , -0.58767126,  0.3832482 ]])

### A.2.5 神奇索引的等价方法：take 和 put

In [82]:
arr=np.arange(10)*100
# array([  0, 100, 200, 300, 400, 500, 600, 700, 800, 900])
inds=[7,1,2,6]
arr[inds] # 从arr序列中依次获取索引为7，1，2，6位置上的元素

array([700, 100, 200, 600])

In [83]:
arr.take(inds)

array([700, 100, 200, 600])

In [85]:
arr.put(inds,42)
# 相当于将arr序列中索引为7，1，2，6位置上的元素用42来替换
arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [87]:
arr.put(inds,[40,41,42,43])
arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

要在别的轴上使用take,需要传递axis关键字

In [88]:
inds=[2,0,2,1]
arr=np.random.randn(2,4)
arr

array([[-0.00258421,  1.40117044,  0.17249806, -0.2259104 ],
       [-0.42279548,  0.81181772, -0.57728944, -1.7101881 ]])

In [89]:
arr.take(inds,axis=1)

array([[ 0.17249806, -0.00258421,  0.17249806,  1.40117044],
       [-0.57728944, -0.42279548, -0.57728944,  0.81181772]])

put不接受axis参数，而是将数组索引到扁平版本(C顺序，一维)

## A.3 广播
广播机制：Numpy可以转换这些形状不同的数组，使它们都具有相同的大小，然后对它们进行运算

In [3]:
import numpy as np
arr=np.arange(5)
arr

array([0, 1, 2, 3, 4])

In [4]:
arr*4

array([ 0,  4,  8, 12, 16])

In [23]:
arr=np.random.randn(4,3)
arr

array([[ 0.05497317, -0.11337856, -0.42908773],
       [-1.15097962,  1.61697728,  0.0708386 ],
       [ 0.87345221,  0.96343496, -0.75677472],
       [-0.11760662,  0.22449413, -0.14855889]])

每一列的均值

In [24]:
arr.mean(0) 

array([-0.08504022,  0.67288195, -0.31589568])

每一列减去相对应的均值

In [26]:
demeaned=arr-arr.mean(0)
demeaned

array([[ 0.14001338, -0.78626051, -0.11319204],
       [-1.06593941,  0.94409533,  0.38673429],
       [ 0.95849243,  0.290553  , -0.44087903],
       [-0.03256641, -0.44838783,  0.16733679]])

In [27]:
arr

array([[ 0.05497317, -0.11337856, -0.42908773],
       [-1.15097962,  1.61697728,  0.0708386 ],
       [ 0.87345221,  0.96343496, -0.75677472],
       [-0.11760662,  0.22449413, -0.14855889]])

In [58]:
row_means=arr.mean(1) # 求出axis=1轴上的均值
row_means

array([-0.1624977 ,  0.17894542,  0.36003748, -0.01389046])

In [31]:
row_means.shape

(4,)

In [33]:
row_means.reshape((4,1))

array([[-0.1624977 ],
       [ 0.17894542],
       [ 0.36003748],
       [-0.01389046]])

In [38]:
demeaned=arr-row_means.reshape((4,1))
demeaned

array([[ 0.21747087,  0.04911915, -0.26659002],
       [-1.32992504,  1.43803186, -0.10810682],
       [ 0.51341473,  0.60339747, -1.1168122 ],
       [-0.10371616,  0.23838459, -0.13466843]])

In [39]:
demeaned.mean(1)

array([-1.85037171e-17,  2.77555756e-17, -7.40148683e-17,  0.00000000e+00])

In [48]:
a=np.arange(24).reshape(3,4,2)
a

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[16, 17],
        [18, 19],
        [20, 21],
        [22, 23]]])

In [52]:
a.mean(0)

array([[ 8.,  9.],
       [10., 11.],
       [12., 13.],
       [14., 15.]])

In [55]:
demean=a+a.mean(0)
demean

array([[[ 8., 10.],
        [12., 14.],
        [16., 18.],
        [20., 22.]],

       [[16., 18.],
        [20., 22.],
        [24., 26.],
        [28., 30.]],

       [[24., 26.],
        [28., 30.],
        [32., 34.],
        [36., 38.]]])

###  A.3.1 在其他轴上广播

In [57]:
arr-arr.mean(1)

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

根据广播规则，“广播维度”在较小的数组中必须为1，在这里显示的行减均值的例子中，这表示重新塑造行意味着形状是(4,1)而不是(4,)

In [59]:
arr-arr.mean(1).reshape(4,1)

array([[ 0.21747087,  0.04911915, -0.26659002],
       [-1.32992504,  1.43803186, -0.10810682],
       [ 0.51341473,  0.60339747, -1.1168122 ],
       [-0.10371616,  0.23838459, -0.13466843]])

使用np.newaxis属性和“完整”切片来插入新轴

In [61]:
arr=np.zeros((4,4))
arr

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [63]:
arr_3d=arr[:,np.newaxis,:]
arr_3d

array([[[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]]])

In [64]:
arr_3d.shape

(4, 1, 4)

In [66]:
arr_1d=np.random.normal(size=3)
arr_1d

array([-0.38233005, -1.55431869,  1.17027235])

In [68]:
arr_1d[:,np.newaxis]

array([[-0.38233005],
       [-1.55431869],
       [ 1.17027235]])

In [69]:
arr_1d[np.newaxis,:]

array([[-0.38233005, -1.55431869,  1.17027235]])

若想在轴2上减去均值

In [71]:
arr=np.random.randn(3,4,5)
arr

array([[[ 6.21067149e-01, -3.58667345e-01,  4.54947130e-01,
         -1.50196339e+00,  1.04035665e+00],
        [ 7.48935127e-01,  1.34055685e+00, -8.90385639e-04,
          8.29291834e-01, -5.30847830e-01],
        [ 5.32540417e-01, -1.17687813e+00, -2.90254072e-01,
         -3.95249422e-01,  1.81190225e-01],
        [-7.15839571e-01,  6.88796033e-01, -2.15914354e-01,
         -7.59371336e-01, -1.15996761e+00]],

       [[ 5.55277404e-01,  4.47489979e-01,  6.98574979e-01,
         -6.46042793e-01, -1.48170474e+00],
        [ 6.88186410e-01, -1.18659067e-01, -2.54292816e-01,
         -1.95314488e+00, -3.60274582e-01],
        [-7.71653684e-01, -2.97334125e-01, -1.98095715e+00,
         -8.13677213e-01, -1.30221228e+00],
        [ 7.74856058e-01, -2.13693793e+00, -9.66369069e-01,
         -7.73626772e-01, -7.83502172e-01]],

       [[-8.46990599e-01,  4.70509662e-01, -3.17792563e-01,
          8.30542479e-01,  1.45763603e+00],
        [ 1.43797162e+00, -8.34838935e-01,  3.04492128e-01,


In [72]:
depth_means=arr.mean(2)
depth_means

array([[ 0.05114804,  0.47740912, -0.2297302 , -0.43245937],
       [-0.08528103, -0.39963699, -1.03316689, -0.77711598],
       [ 0.318781  ,  0.50787214,  0.68745559,  0.75012726]])

In [73]:
depth_means.shape

(3, 4)

In [79]:
demeaned=depth_means[:,:,np.newaxis]
demeaned

array([[[ 0.05114804],
        [ 0.47740912],
        [-0.2297302 ],
        [-0.43245937]],

       [[-0.08528103],
        [-0.39963699],
        [-1.03316689],
        [-0.77711598]],

       [[ 0.318781  ],
        [ 0.50787214],
        [ 0.68745559],
        [ 0.75012726]]])

In [80]:
depth_means[:,:,np.newaxis].shape

(3, 4, 1)

In [81]:
demeaned.mean(2)

array([[ 0.05114804,  0.47740912, -0.2297302 , -0.43245937],
       [-0.08528103, -0.39963699, -1.03316689, -0.77711598],
       [ 0.318781  ,  0.50787214,  0.68745559,  0.75012726]])

### A.3.2 通过广播设定数组的值

控制算数运算的相同广播规则也适用于通过数组索引设置值

In [85]:
arr=np.zeros((4,3))
arr

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [89]:
arr[:]=5 # 将数组中的值全替换为5
arr

array([[5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.]])

将数值的一维数组设置到数组的列中，只要是形状兼容就可以做到这一点

In [90]:
col=np.array([1.28,-0.42,0.44,1.6])
col

array([ 1.28, -0.42,  0.44,  1.6 ])

In [93]:
arr[:]=col[:,np.newaxis]
arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [95]:
arr[:2]=[[-1.37],[0.509]]
# 将前两行的值分别设置为-1.37,0.509
arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])

## A.4 高阶 ufunc用法

In [27]:
# 使用np.add.reduce是对数组中元素进行相加的另一种方法
arr=np.arange(10)
np.add.reduce(arr) # == add.sum()

45

np.logical_and(x,y)对于两个参数进行逻辑与运算

In [38]:
arr=np.arange(25).reshape(5,5)

In [43]:
arr[::2].sort(1) # 步长为2采样

In [46]:
arr[:,:-1]
# 取完每一行，列的最后一个不取

array([[ 0,  1,  2,  3],
       [ 5,  6,  7,  8],
       [10, 11, 12, 13],
       [15, 16, 17, 18],
       [20, 21, 22, 23]])

In [47]:
arr[:,1:] 
# 取完每一行，列的第一个不取

array([[ 1,  2,  3,  4],
       [ 6,  7,  8,  9],
       [11, 12, 13, 14],
       [16, 17, 18, 19],
       [21, 22, 23, 24]])

In [48]:
arr[:,:-1]<arr[:,1:] 

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [49]:
np.logical_and.reduce(arr[:,:-1]<arr[:,1:],axis=1) # 检查每一行是否都满足arr[:,:-1]<arr[:,1:] 

array([ True,  True,  True,  True,  True])

In [51]:
arr=np.arange(15).reshape((3,5))
arr

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [53]:
np.add.accumulate(arr,axis=1) # 逐行累加

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]], dtype=int32)

In [54]:
arr=np.arange(3).repeat([1,2,2])
arr

array([0, 1, 1, 2, 2])

In [57]:
np.multiply.outer(arr,np.arange(5))
# [0,1,1,2,2] --> 数组中的每个值与[0,1,2,3,4]相乘(逐行)
# arr=np.multiply.outer(arr1,arr2)
# arr[i,j]=arr1[i]*arr2[j]

array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

outer的输出维度等于输入的维度总和

In [60]:
x,y=np.random.randn(3,4),np.random.randn(5)
result=np.subtract.outer(x,y)
result.shape

(3, 4, 5)

In [61]:
arr=np.arange(10)
np.add.reduceat(arr,[0,5,8])
# [0:5] [5:8] [8:]

array([10, 18, 17], dtype=int32)

In [62]:
arr=np.multiply.outer(np.arange(4),np.arange(5))
arr

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

In [64]:
np.add.reduceat(arr,[0,2,4],axis=1)
#       [ 0,  0,  0], [0+0,0+0,0+0]
#       [ 1,  5,  4], [0+1,2+3,4]
#       [ 2, 10,  8], [0+2,4+6,8]
#       [ 3, 15, 12]  [0+3,6+9,12]

array([[ 0,  0,  0],
       [ 1,  5,  4],
       [ 2, 10,  8],
       [ 3, 15, 12]], dtype=int32)

### A.4.2 使用Python编写新的ufunc方法
numpy.frompyfunc函数接收一个具有特定数字输入和输出的函数

In [72]:
def add_elements(x,y):
    return x+y
add_them=np.frompyfunc(add_elements,2,1)

In [73]:
add_them(np.arange(8),np.arange(8))
# [0+0,1+1,2+2,3+3,4+4,5+5,6+6,7+7]

array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

numpy.vectorize允许指定输出的类型

In [75]:
add_them=np.vectorize(add_elements,otypes=[np.float64]) # otypes=[np.float64] 指定输出的类型
add_them(np.arange(8),np.arange(8))

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

In [76]:
arr=np.random.randn(10000)

检查一段Python语句的执行时间

In [77]:
%timeit add_them(arr,arr)

3.96 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
%timeit np.add(arr,arr)

10.2 µs ± 854 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## A.5 结构化和记录数组
ndarray是一个同构数据的容器(数组中的元素都是同一类型，在它所表示的内存块中，各元素占用的字节数相同，具体由dtype决定)

In [2]:
import numpy as np
dtype=[('x',np.float64),('y',np.float32)]
sarr=np.array([(1.5,6),(np.pi,-2)],dtype=dtype)
sarr

array([(1.5       ,  6.), (3.14159265, -2.)],
      dtype=[('x', '<f8'), ('y', '<f4')])

In [3]:
sarr[0]

(1.5, 6.)

In [4]:
sarr[0]['y']

6.0

字段名称存储在dtype,names属性中，当你访问结构化数组中的字段时，将返回数据的分步视图，因此不会复制任何内容

In [5]:
sarr['x']

array([1.5       , 3.14159265])

### A.5.1 嵌套dtype和多维字段

In [6]:
dtype=[('x',np.int64,3),('y',np.int32)] # 
arr=np.zeros(4,dtype=dtype)
arr

array([([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0)],
      dtype=[('x', '<i8', (3,)), ('y', '<i4')])

In [7]:
arr[0] # 可以把arr看作一个表格，arr[0]访问第一行

([0, 0, 0], 0)

In [8]:
arr[0]['x'] # 访问第一行的x字段/列，长度为3的一维数组

array([0, 0, 0], dtype=int64)

In [9]:
arr['x'] #访问arr的x字段，2维数组 

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

In [10]:
# 定义各字段/列的类型，x字段又分了两个小字段
dtype=[('x',[('a','f8'),('b','f4')]),('y',np.int32)]
data=np.array([((1,2),5),((3,4),6)],dtype=dtype)
data['x'] # 访问x字段

array([(1., 2.), (3., 4.)], dtype=[('a', '<f8'), ('b', '<f4')])

In [11]:
data[0] # 访问第一行/第一个记录

((1., 2.), 5)

In [12]:
data['y'] # 访问y字段

array([5, 6])

In [13]:
data['x']['a'] # 访问x字段下的a字段

array([1., 3.])

### A.5.2 为什么要使用结构化数组

## A.6 更多关于排序的内容
ndarray的sort实例方法是一种原位排列，意味着数组的内容进行了重排列，而不是生成了一个新的数组

In [1]:
import numpy as np
arr=np.random.randn(6)
arr

array([ 1.34846835,  0.0725504 ,  1.86823428, -1.07552982, -0.23566821,
        0.15158749])

In [3]:
arr.sort()
arr

array([-1.07552982, -0.23566821,  0.0725504 ,  0.15158749,  1.34846835,
        1.86823428])

在进行数组原位排序时，如果数组是不同的ndarray的视图的话，原始数组将会被改变

In [8]:
arr=np.random.randn(3,5)
arr

array([[ 0.74312351, -1.1994112 ,  1.37017219, -0.40621883,  0.45454458],
       [-0.44789372, -0.70522404,  0.49712325,  0.28699005,  1.14787119],
       [-0.75451477, -1.3963117 , -0.89722156,  0.23841306, -0.5273099 ]])

In [11]:
arr[:,0].sort() # 对第一列的值原位排序(默认升序)
arr

array([[-0.75451477, -1.1994112 ,  1.37017219, -0.40621883,  0.45454458],
       [-0.44789372, -0.70522404,  0.49712325,  0.28699005,  1.14787119],
       [ 0.74312351, -1.3963117 , -0.89722156,  0.23841306, -0.5273099 ]])

numpy.sort 产生的是一个数组的新的，排序后的副本

In [12]:
arr=np.random.randn(5)
arr

array([ 0.92435754, -0.10490516, -1.57031842, -1.03392492,  1.11743119])

In [15]:
np.sort(arr) # 新的副本

array([-1.57031842, -1.03392492, -0.10490516,  0.92435754,  1.11743119])

In [16]:
arr # 原始数组不变

array([ 0.92435754, -0.10490516, -1.57031842, -1.03392492,  1.11743119])

In [17]:
arr=np.random.randn(3,5)
arr

array([[-0.22590626,  0.23816341, -0.17734563,  0.4450031 , -1.93625942],
       [-1.45192148, -0.82699796,  1.18746417, -0.76104882, -0.15645459],
       [ 1.66570441, -1.88381963, -0.04096219,  0.5245202 ,  1.90044849]])

In [19]:
arr.sort(axis=1) 
# 按 axis=1 排序
arr

array([[-1.93625942, -0.22590626, -0.17734563,  0.23816341,  0.4450031 ],
       [-1.45192148, -0.82699796, -0.76104882, -0.15645459,  1.18746417],
       [-1.88381963, -0.04096219,  0.5245202 ,  1.66570441,  1.90044849]])

In [21]:
arr[:,::-1]

array([[ 0.4450031 ,  0.23816341, -0.17734563, -0.22590626, -1.93625942],
       [ 1.18746417, -0.15645459, -0.76104882, -0.82699796, -1.45192148],
       [ 1.90044849,  1.66570441,  0.5245202 , -0.04096219, -1.88381963]])

### A.6.1 间接排序： argsort和lexsort

利用argsort()函数对数组排序，实际上返回的是数组元素从小到大位置的索引

In [23]:
values=np.array([5,0,1,3,2])
indexer=values.argsort()
indexer # 对values排序的值所对应的索引

array([1, 2, 4, 3, 0], dtype=int64)

In [25]:
values[indexer] # indexer下的值

array([0, 1, 2, 3, 5])

In [27]:
arr=np.random.randn(3,5)
arr[0]=values
# 将arr中的第一行值用values替换
arr

array([[ 5.        ,  0.        ,  1.        ,  3.        ,  2.        ],
       [-0.41185354, -1.63563781,  0.03253898,  0.12260932,  1.40180929],
       [-1.19470505, -0.17046939,  0.62209543,  1.33757477, -0.10672115]])

In [29]:
arr[:,arr[0].argsort()] # 输出每一行，但是将第一排排序

array([[ 0.        ,  1.        ,  2.        ,  3.        ,  5.        ],
       [-1.63563781,  0.03253898,  1.40180929,  0.12260932, -0.41185354],
       [-0.17046939,  0.62209543, -0.10672115,  1.33757477, -1.19470505]])

lexsort类似于argsort，但它对多键数组执行间接字典排序

In [30]:
first_name=np.array(['Bob','Jane','Steve','Bill','Barbara']) # 第二步
last_name=np.array(['Jones','Arnold','Arnold','Jones','Walters']) # 第一步
sorter=np.lexsort((first_name,last_name))
sorter

array([1, 2, 3, 0, 4], dtype=int64)

In [31]:
zip(last_name[sorter],first_name[sorter])

lexsort用于排序数据的键的顺序从传递的最后一个数组开始，这里，last_name在first_name之前使用

### A.6.2 其他的排序算法

In [5]:
import numpy as np
values=np.array(['2:first','2:second','1:first','1:second','2:third'])
key=np.array([2,2,1,1,1])
indexer=key.argsort(kind='mergesort')
indexer

array([2, 3, 4, 0, 1], dtype=int64)

In [6]:
values.take(indexer)

array(['1:first', '1:second', '2:third', '2:first', '2:second'],
      dtype='<U8')

### A.6.3 数组的部分序列

In [7]:
np.random.seed(12345)
arr=np.random.randn(20)
arr

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057,
        1.39340583,  0.09290788,  0.28174615,  0.76902257,  1.24643474,
        1.00718936, -1.29622111,  0.27499163,  0.22891288,  1.35291684,
        0.88642934, -2.00163731, -0.37184254,  1.66902531, -0.43856974])

In [9]:
np.partition(arr,3) # 结果中的前三个元素是最小的三个值，并不是特定的顺序

array([-2.00163731, -1.29622111, -0.5557303 , -0.51943872, -0.37184254,
       -0.43856974, -0.20470766,  0.28174615,  0.76902257,  0.47894334,
        1.00718936,  0.09290788,  0.27499163,  0.22891288,  1.35291684,
        0.88642934,  1.39340583,  1.96578057,  1.66902531,  1.24643474])

np.argpartition 类似于 numpy.argsort排序，它返回的是将数据重新排列为等价顺序的索引

In [11]:
indices=np.argpartition(arr,3)
indices

array([16, 11,  3,  2, 17, 19,  0,  7,  8,  1, 10,  6, 12, 13, 14, 15,  5,
        4, 18,  9], dtype=int64)

In [12]:
arr.take(indices)

array([-2.00163731, -1.29622111, -0.5557303 , -0.51943872, -0.37184254,
       -0.43856974, -0.20470766,  0.28174615,  0.76902257,  0.47894334,
        1.00718936,  0.09290788,  0.27499163,  0.22891288,  1.35291684,
        0.88642934,  1.39340583,  1.96578057,  1.66902531,  1.24643474])

### A.6.4 numpy.searchsorted:在已排序的数组寻找元素

searchsorted是一个数组方法，它对已排序数组执行二分搜索，返回数组中需要插入值的位置以保持排序

In [14]:
arr=np.array([0,1,7,12,15])
arr.searchsorted(9) # 返回9插入的位置索引

3

也可以传递一个值数组来获取一个索引数组

In [19]:
arr.searchsorted([0,8,11,16])
# 对于0元素，返回0，因为默认行认为时返回一组相等值左侧的索引

array([0, 3, 3, 5], dtype=int64)

In [20]:
data=np.floor(np.random.uniform(0,10000,size=50))
bins=np.array([0,100,1000,5000,10000])
data

array([9940., 6768., 7908., 1709.,  268., 8003., 9037.,  246., 4917.,
       5262., 5963.,  519., 8950., 7282., 8183., 5002., 8101.,  959.,
       2189., 2587., 4681., 4593., 7095., 1780., 5314., 1677., 7688.,
       9281., 6094., 1501., 4896., 3773., 8486., 9110., 3838., 3154.,
       5683., 1878., 1258., 6875., 7996., 5735., 9732., 6340., 8884.,
       4954., 3516., 7142., 5039., 2256.])

In [23]:
labels=bins.searchsorted(data)
labels # 返回每个数据点属于哪个区间的标签

array([4, 4, 4, 3, 2, 4, 4, 2, 3, 4, 4, 2, 4, 4, 4, 4, 4, 2, 3, 3, 3, 3,
       4, 3, 4, 3, 4, 4, 4, 3, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 4, 4, 3], dtype=int64)

可以和pandas的groupby一起被用于分箱数据

In [24]:
import pandas as pd
pd.Series(data).groupby(labels).mean() # 按标签区间分组，求均值

2     498.000000
3    3064.277778
4    7389.035714
dtype: float64

## A.7 使用Numba编写快速NumPy函数

定义一个使用for循环计算表达式(x-y).mean()的值

In [32]:
import numpy as np
def mean_distance(x,y):
    nx=len(x)
    result=0.0
    count=0
    for i in range(nx):
        result+=x[i]-y[i]
        count+=1
    return result/count

In [33]:
x=np.random.randn(10000000)
y=np.random.randn(10000000)

这个函数非常慢

In [34]:
%timeit mean_distance(x,y)

11.2 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit (x-y).mean()

66.9 ms ± 7.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


使用numba.jit函数将这个函数编译成Numba函数

In [36]:
import numba as nb
numba_mean_distance=nb.jit(mean_distance)

也可以写成装饰器的形式

In [38]:
@nb.jit
def mean_distance(x,y):
    nx=len(x)
    result=0.0
    count=0
    for i in range(nx):
        result+=x[i]-y[i]
        count+=1
    return result/count

In [39]:
%timeit numba_mean_distance(x,y)

27.4 ms ± 493 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
from numba import float64,njit
@njit(float64(float64[:],float64[:]))
def mean_distnace(x,y):
    return(x-y).mean()

### A.7.1 使用Numba创建自定义numpy.ufunc对象
numba.vectorize 函数创建了编译好的NumPy ufunc

In [74]:
import numba as nb
from numba import vectorize
@vectorize
def nb_add(x,y):
    return x+y

In [75]:
x=np.arange(10)
nb_add(x,x)
# [0+0,1+1,2+2,3+3,4+4,5+5,6+6,7+7,8+8,9+9]

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18], dtype=int64)

In [81]:
nb_add.accumulate(x,0)

ValueError: could not find a matching type for nb_add.accumulate, requested type has type code 'l'

## A.8 高阶数组的输入和输出

np.save和np.load，它们用于在磁盘上以二进制的格式存储数组

### A.8.1 内存映射文件
内存映射文件是一种与磁盘上的二进制数据交互的方法，就像是它存储在内存数组中一样

NumPy实现了一个memmap对象，它是ndarray型的，允许对大型文件以小堆栈的方式进行读取和写入，而无须将整个数组载入内存

要创建一个新的内存映射，使用np.memmap并传入文件路径,dtype,shape和文件模式

In [3]:
import numpy as np
mmap=np.memmap('mymmap',dtype='float64',mode='w+',shape=(10000,10000))
mmap

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

对memmap切片返回的是硬盘上数据的视图

In [6]:
section=mmap[:5] # 切片取前五行
section

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

如果你将数据赋值给这些切片，它将会在内存中缓存(类似于一个Python文件对象)，但你可以调用flush将数据写入硬盘

In [7]:
section[:]=np.random.randn(5,10000)
mmap.flush()
mmap

memmap([[ 2.29175416,  0.3609065 ,  0.3426645 , ...,  1.50901946,
         -0.145749  , -1.40765696],
        [-0.80392839, -0.86160891,  0.0174384 , ...,  0.36083377,
         -0.63350301,  0.28864519],
        [ 0.14668864, -0.73272069,  0.60495037, ..., -2.47503953,
          0.21110734, -1.99025864],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

每当内存映射超出范围并且被垃圾收集时，任何更改都将刷新到磁盘。打开已经存在的内存映射时，你仍然必须指定dtype和shape，因为文件只是磁盘上没有元数据的二进制数据块

In [10]:
mmap=np.memmap('mymmap',dtype='float64',shape=(10000,10000))
mmap

memmap([[ 2.29175416,  0.3609065 ,  0.3426645 , ...,  1.50901946,
         -0.145749  , -1.40765696],
        [-0.80392839, -0.86160891,  0.0174384 , ...,  0.36083377,
         -0.63350301,  0.28864519],
        [ 0.14668864, -0.73272069,  0.60495037, ..., -2.47503953,
          0.21110734, -1.99025864],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

### A.8.2 HDF5和其他数组存储选择

## A.9 性能技巧

### A.9.1 连续内存的重要性

访问连续内存块的操作(例如：将C的顺序数组的行相加)通常是最快的
如果说数组的内存布局是连续的，就意味着这些元素按照它们在Fortran顺序(列方向)或C顺序(行方向)排序中出现在数组中的次序存储在内存中
默认情况下，NumPy数组创建为C顺序连续或只是简单连续，因此，一个列方向的数组，例如C顺序数组的转置数组，可以称为是Fortran顺序连续的
这些属性都是可以通过ndarray的flags属性来检查

In [11]:
arr_c=np.ones((1000,1000),order='C')
arr_f=np.ones((1000,1000),order='F')

In [12]:
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [13]:
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

对这些数组的行及逆行加和，在理论上arr_c是比arr_f更快，因为arr_c的行在内存中是连续的

In [14]:
%timeit arr_c.sum(1)

1.43 ms ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit arr_f.sum(1)

1.41 ms ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


如果你的数组没有所需的内存顺序,则可以使用copy并传递'C'和'F'

In [16]:
arr_f.copy('C').flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

在数组上构建视图的时候，请记住结果并不能保证是连续的

In [17]:
arr_c[:50].flags.contiguous # 检查是否连续

True

In [18]:
arr_c[:,:50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False